## **Загрузка**

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

# Визуализация
import matplotlib.pyplot as plt

# Машинное обучение
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer

# Нейросети
import torch
import torchvision
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM, SpatialDropout1D

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/CSV

Mounted at /content/drive/
/content/drive/My Drive/CSV


In [3]:
data = pd.read_csv('news.csv', usecols=range(1,3))

In [4]:
data.info(show_counts = 1) 
print('\nКоличество повторяющихся строк —', data.duplicated().sum())
display(data.head(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19462 entries, 0 to 19461
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   news    19462 non-null  object 
 1   source  18360 non-null  float64
dtypes: float64(1), object(1)
memory usage: 304.2+ KB

Количество повторяющихся строк — 29


,news,source
0,Официальный аккаунт PlayStation опубликовал т...,1.0
1,Китайская компания Mobvoi опубликовала на офи...,1.0


* Удалить дубликаты.
* Обработать текст для подачи на вход нейронной сети.

# **Предобработка**

## Удаление дубликатов

In [5]:
data_cleaned = data.drop_duplicates()

In [6]:
data_cleaned.duplicated().sum()

0

## Обработка текста

In [7]:
# Максимальное количество слов в новости
maxlen = max(data_cleaned.news.apply(lambda x: len(x.split())))
maxlen

6495

In [8]:
# Частота самого упоминаемого слова
max_features = data_cleaned.news.str.split(expand=True).stack().value_counts().max()
max_features

175854

In [9]:
tokenizer = Tokenizer(num_words = 20000)

In [10]:
sentences = data_cleaned['news'].values

In [11]:
y = data_cleaned['source'].values

In [12]:
sentences_train, sentences_test, y_train, y_test =\
 train_test_split(sentences, y, test_size=0.25, random_state=1000)

In [13]:
tokenizer.fit_on_texts(sentences_train)

In [14]:
X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

In [15]:
vocab_size = len(tokenizer.word_index) + 1

In [16]:
print(sentences_train[2])
print(X_train[2])

 Стриминговый сервис Spotify в настоящее время испытывает проблемы в работе: пользователи жалуются, что не загруженные в офлайн треки воспроизводятся несколько секунд, после чего приостанавливаются. Сервис Downdetector, фиксирующий сбои сервисов, передаёт о почти 6000 отчётах о перебоях в работе Spotify. Судя по карте, чаще всего пользователи регистрируют недоступность стриминговой платформы в Европе и США, но есть отчёты и из Африки, Австралии, Южной Америки. Есть проблемы и у российских клиентов. В Twitter-профиле Spotify Status появилось сообщение следующего содержания: «Мы в курсе некоторых проблем, наблюдаемых в настоящее время, и проверяем их». Spotify официально появился в России и ряде других постсоветских стран в середине июля. 
[4088, 201, 1407, 1, 1449, 69, 11462, 468, 1, 584, 143, 3857, 5, 6, 14002, 1, 4270, 7524, 117, 3263, 66, 453, 201, 5472, 5420, 366, 17641, 15, 346, 5719, 15, 1, 584, 1407, 1547, 8, 3939, 1049, 103, 143, 15605, 539, 1, 1034, 2, 72, 18, 65, 16091, 2, 10,

In [17]:
print(sentences_test[2])
print(X_test[2])

 Испанский парламент принял новый закон об интеллектуальной собственности, обязывающий агрегаторы новостей платить издателям за контент, сообщила сегодня испанская газета El Pais. Интернет-компании должны будут делать это всякий раз, когда в результатах поиска агрегатора появляется та или иная новость. За принятие этого закона, по данным El Pais, проголосовали 172 депутата нижней палаты парламента, против были 144 депутата, трое воздержались. Закон начнет действовать с 1 января 2015 г. Однако размер компенсации в нем не указан.  Правительство Испании надеется, что этот закон ослабит интернет-пиратство, улучшит систему управления авторскими правами и заставит такие агрегаторы новостей, как Google News, компенсировать СМИ ущерб от использования их контента, отмечает газета.  Многие называют этот закон «налогом на Google», который является самым популярным поисковиком в Испании. По данным StatCounter, его доля на конец сентября составляла 95,3%. В Испании эту инициативу чаще называют AEDE

In [18]:
for word in ['в', 'об', 'сервис', 'парламент']:
  print('{}: {}'.format(word, tokenizer.word_index[word]))

в: 1
об: 85
сервис: 201
парламент: 10873


In [19]:
data_cleaned = data_cleaned.copy()
data_cleaned['news_numeric'] = tokenizer.texts_to_sequences(data_cleaned.news)

In [20]:
data_cleaned.head()

,news,source,news_numeric
0,Официальный аккаунт PlayStation опубликовал т...,1.0,"[1052, 1137, 734, 674, 2384, 1077, 202, 122, 3..."
1,Китайская компания Mobvoi опубликовала на офи...,1.0,"[2133, 35, 17215, 1771, 3, 697, 285, 3389, 136..."
2,Практически во всех странах мира введены огра...,0.0,"[418, 74, 134, 588, 562, 10341, 882, 1822, 4, ..."
3,"Депутат Госдумы, единоросс Антон Горелкин вне...",2.0,"[3323, 2260, 2726, 8254, 7043, 1, 13557, 7598,..."
4,Совет директоров «Почты России» проголосовал ...,2.0,"[1515, 885, 4473, 1257, 12, 9075, 7200, 3433, ..."


In [21]:
data_cleaned = data_cleaned.copy()
data_cleaned['news_numeric'] = data_cleaned['news_numeric'].apply(lambda x: np.array(x))

# **Расчёты**

In [22]:
#  Необходимое количество предсказаний
len(data_cleaned.query('source != source'))

1102

In [23]:
X = data_cleaned.query('source == source')['news_numeric']
y = data_cleaned.query('source == source')['source']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) 

In [25]:
X_train = [torch.from_numpy(X_train[x]) for x in X_train.index]

In [26]:
X_test = [torch.from_numpy(X_test[x]) for x in X_test.index]

In [27]:
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [28]:
y_train = tf.stack(y_train)
y_test = tf.stack(y_test)

In [29]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2)) 
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
len(X_train[0])

6495

In [31]:
model.fit(X_train, y_train, batch_size=64, epochs=6,
          validation_data=(X_test, y_test), verbose=2)

Epoch 1/6
230/230 - 4561s - loss: -7.4679e-02 - accuracy: 0.6455 - val_loss: -4.4169e-01 - val_accuracy: 0.6417 - 4561s/epoch - 20s/step
Epoch 2/6
230/230 - 4566s - loss: -3.4781e+00 - accuracy: 0.7317 - val_loss: -7.1038e+00 - val_accuracy: 0.7597 - 4566s/epoch - 20s/step
Epoch 3/6
230/230 - 4678s - loss: -6.5126e+00 - accuracy: 0.7318 - val_loss: -9.1115e+00 - val_accuracy: 0.7281 - 4678s/epoch - 20s/step
Epoch 4/6
230/230 - 5022s - loss: -1.1516e+01 - accuracy: 0.7647 - val_loss: -1.4025e+01 - val_accuracy: 0.7497 - 5022s/epoch - 22s/step
Epoch 5/6
230/230 - 4581s - loss: -1.2504e+01 - accuracy: 0.7655 - val_loss: -8.3529e+00 - val_accuracy: 0.6957 - 4581s/epoch - 20s/step
Epoch 6/6
230/230 - 4577s - loss: -1.6315e+01 - accuracy: 0.7849 - val_loss: -1.2026e+01 - val_accuracy: 0.7208 - 4577s/epoch - 20s/step


In [32]:
test = data_cleaned.query('source != source')

In [33]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1102 entries, 18360 to 19461
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   news          1102 non-null   object 
 1   source        0 non-null      float64
 2   news_numeric  1102 non-null   object 
dtypes: float64(1), object(2)
memory usage: 34.4+ KB


In [34]:
test_news_numeric = test[['news_numeric']].to_numpy()

In [35]:
test_news_numeric = [torch.from_numpy(test['news_numeric'][x]) for x in (test.index)]

In [36]:
test = test.copy()
test['source'] = tf.stack(test['source'])

In [37]:
test_news_numeric = pad_sequences(test_news_numeric, maxlen=maxlen)

In [38]:
prediction = model.predict(test_news_numeric)

35/35 [==============================] - 55s 2s/step


___

# **Выгрузка**

In [39]:
len(prediction)

1102

In [40]:
rt = pd.Series(prediction.ravel())

In [41]:
rt = np.floor(rt)

In [42]:
rt.to_csv('news_matveeva-a.csv', index=False, header=False)